In [98]:
from sklearn import metrics
import numpy as np
import pandas as pd
from scipy import stats as st

In [99]:
# re-construct the parameters of the training and test data set
num_features = 50
num_indv = 40
num_obsr = 40
flag = "bcd"

alpha_b = 1
np.random.seed(1)  # this is fixed
ind = 10 # only keep 10 non-zero effects
intersect = 5
bi = np.random.uniform(1,0,size=ind)
bo = np.random.uniform(1,0,size=ind)
bi = np.append(bi,np.zeros(num_features - ind))
if num_features > intersect + ind:
    bo = np.append(np.zeros(intersect),bo)
    bo = np.append(bo,np.zeros(num_features - intersect - ind))
else:
    bo = np.append(np.zeros(intersect),bo)[:num_features]
halfInd = int(ind/2)
bi[ind:] = 0
bo[:halfInd] = 0
bo[(halfInd + ind):] = 0
fixedEff = [1,2,3,-1,-2,-3,7,10]
fixedEff = np.append(fixedEff,np.zeros([num_features - len(fixedEff)]))

# get theta
thetai = np.zeros([num_indv,num_features])
for i in range(num_features):
    if bi[i] > 0:
        thetai[:,i] = st.laplace.rvs(loc = 0,scale = bi[i],size=num_indv)
thetao = np.zeros([num_obsr,num_features])
for i in range(num_features):
    if bo[i] > 0:
        thetao[:,i] = st.laplace.rvs(loc = 0,scale = bo[i],size=num_obsr)
    
def getDataset(dataFr):
    features = list(dataFr.iloc[:,np.arange(num_features)].values)
    data = pd.DataFrame()
    data['feature'] = features
    data['iid'] = dataFr['iid']
    data['oid'] = dataFr['oid'] + num_indv
    data['label'] = dataFr['label']
    return data

In [100]:
%run "LMLFM.ipynb"
filePath = ''
trainFname = filePath + 'train.csv'
testFname = filePath + 'test.csv'
trainFr = pd.read_csv(trainFname)
testFr = pd.read_csv(testFname)
train = getDataset(trainFr)
test = getDataset(testFr)

lmlfm = LMLFM(num_users, num_observations, num_features,verbose = True, iterations = 50,alpha = 15 / train.label.var(),epsilon = 1e-2)
theta = lmlfm.fit(train,test)
_,loss, r = lmlfm.evaluate(test)

initalization ...
Iter    TrLL    TeLL   TrRMSE   TeRMSE   TrR2   TeR2
0    -7.0212    -2.8525    14.4877    14.1848    -0.0003    -0.0038
1    -3.5307    -3.7265    2.2742    12.4995    0.9754    0.2205
2    -3.2669    -3.3656    1.7334    11.4739    0.9857    0.3432
3    -3.1331    -3.1432    1.7268    10.8015    0.9858    0.4179
4    -3.0538    -2.9844    1.7038    10.1488    0.9862    0.4861
5    -3.0087    -2.8899    1.6901    9.7222    0.9864    0.5284
finish initialization...
Iter    TrLL    TeLL   TrRMSE   TeRMSE   TrR2   TeR2
0    -0.7481    -2.9395    1.6901    9.7222    0.9864    0.5284
1    0.0617    -1.2558    1.8180    8.7177    0.9842    0.6208
2    2.5720    2.3624    2.2955    7.6084    0.9749    0.7112
3    7.7521    8.1550    2.7993    6.0946    0.9627    0.8147
4    13.7321    14.3673    3.0210    4.6576    0.9565    0.8918
5    18.1966    18.8855    3.0106    3.9392    0.9568    0.9226
6    19.1703    19.8582    2.9598    3.7781    0.9582    0.9288
7    19.3948    

In [101]:
# the r2 score
print('r2')
print(r)

# estimated bi
estbi = np.round(lmlfm.b_i,5)
msebi = np.mean(np.square(estbi[:ind] - bi[:ind]))
print('mse of bi')
print(msebi)

# estimated bo
estbo = np.round(lmlfm.b_o,5)
msebo = np.mean(np.square(estbo[halfInd:(halfInd + ind)] - bo[halfInd:(halfInd + ind)]))
print('mse of bo')
print(msebo)

# estimated fixed effects
estbeta = np.round(lmlfm.fixedEffects(),5)
msebeta = np.mean(np.square(estbeta - fixedEff))
print('mse of beta')
print(msebeta)

fp = []
fn = []
hitfp = 0
hitfn = 0
for j in range(len(estbeta)):
    if estbeta[j] != 0 and fixedEff[j] == 0:
        hitfp += 1
    if estbeta[j] == 0 and fixedEff[j] != 0:
        hitfn += 1

print('fp')
print(hitfp)
print('fn')
print(hitfn)

r2
0.9255601830524917
mse of bi
0.06754934704253371
mse of bo
0.25914323236705833
mse of beta
0.06228817900800001
fp
0
fn
0
